<img src="Dau.png">

#### MASEF, University Paris-Dauphine 2021:   Bryan Delamour 

# Deep Hedging Under Rough Volatility


## $1)$ Hedging

In [ ]:
import numpy as np
from cmath import * 
from time import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pickle
import os
from termcolor import colored

### European Call

Parameters: 

Strike $K = 100$


Spot price $S_0 = 100 $

Maturity $ T = 30/365 $ 


### Hedging parameters

Daily Hedge

Number of hedging instruments $ d=2 $ 

Instruments = asset price, volatility 

In [ ]:
Product="European Call"
K=100 #strike
S_0=100 # spot price
T=30/365 #Maturity
d=2 # nb of hedging instruments (S,V)
steps=30 # Daily Hedging

## $2)$ Fully Recurrent Neural Network LSTM

## Network architecture


<img src="fRNN_LSTM.png">


In [ ]:
# Fully recurrent LSTM Neural Network with:

    # Batch Normalization
    # sigmoid activation
    # 2 hidden Layers
    
hidden_dim=12
    
class fRNN_LSTM(nn.Module):
    
    def __init__(self):
        
        super(fRNN_LSTM, self).__init__()

        self.lstm_layer1 = nn.LSTM(d, hidden_dim, 1, batch_first=True)#input data is of shape (batch_size, seq_len, features) then you need batch_first=True

        self.lstm_layer2 = nn.LSTM(2*d+4*hidden_dim, hidden_dim, 2, batch_first=True)

        self.output_layer=nn.Linear(hidden_dim, d)

        self.sigmoid=nn.Sigmoid()

        self.BN1= nn.BatchNorm1d(d)

        self.LayerN= nn.LayerNorm(hidden_dim)

        self.relu=nn.ReLU()


    def forward(self, S):
        
        batch_size, steps, _ = S.shape
        output=torch.zeros(S.shape)

        SV0=S[:,0,:].view((batch_size, 1, 2))
        
        delta, h_c_n =self.lstm_layer1(SV0)#input layer t0
        
        delta=self.output_layer(delta) #output layer t0

        deltaS=self.BN1(delta[:,:,:1])
        deltaV=self.BN1(delta[:,:,1:])
        
        deltaS =self.sigmoid(deltaS) #sigmoid delta >=0
        deltaV =self.sigmoid(deltaV)


        output[:,0,:1]=deltaS[:,0,:] # append delta0 list 
        output[:,0,1:]=deltaV[:,0,:]
        
        hn1=torch.reshape( h_c_n[0], (batch_size, 1, hidden_dim) )
        cn1=torch.reshape( h_c_n[1], (batch_size, 1, hidden_dim) )


        hn1=self.LayerN(hn1)
        cn1=self.LayerN(cn1)

        hn2=torch.zeros((batch_size, 1, hidden_dim))
        cn2=torch.zeros((batch_size, 1, hidden_dim))

        for i in range(1,steps):
            
            SVt=S[:,i,:].view((batch_size, 1, 2)) #reshape Sti
            
            St=self.BN1(SVt[:,:,:1]) #BN St
            Vt=self.BN1(SVt[:,:,1:]) #BN Vt

            y=torch.cat((St,Vt,deltaS,deltaV,hn1,cn1,hn2,cn2),dim=2)      #input (Sti,Vti deltaS ti-1,deltaV ti-1,h_c_n)

            delta, h_c_n=self.lstm_layer2(y) #input layer (Sti,delta ti-1,h_c_n)

            delta =self.output_layer(delta) #output delta ti 

            deltaS=self.BN1(delta[:,:,:1])  #BN delta S delta V
            deltaV=self.BN1(delta[:,:,1:])

            deltaS=self.sigmoid(deltaS) #sigmoid delta>=0
            deltaV=self.sigmoid(deltaV)

            output[:,i,:1]=deltaS[:,0,:] # append delta ti list
            output[:,i,1:]=deltaV[:,0,:] 
            

            hn1=torch.reshape( h_c_n[0], (batch_size, 2, hidden_dim) )
            hn1=hn1[:,1:,:]
            hn2=hn1[:,:1,:]

            hn1=self.LayerN(hn1)
            hn2=self.LayerN(hn2)

            cn1=torch.reshape( h_c_n[1], (batch_size, 2, hidden_dim) )
            cn1=cn1[:,1:,:]
            cn2=cn1[:,:1,:]

            cn1=self.LayerN(cn1)
            cn2=self.LayerN(cn2)

        return output #return delta list
      

In [ ]:
fRnn_LSTM=fRNN_LSTM()
print(fRnn_LSTM)

### Data importation

In [ ]:
# Data importation For our rBergomi class

S1_training_File = "S1_training_data.pkl"  
p_0_training_File = "p_0_training_data.pkl"
fRnn_LSTM_model_File = "fRnn_LSTM_model.pkl"  

V1_training_File= "V1_training_data.pkl"

with open(S1_training_File, 'rb') as file:  
    S1_train = pickle.load(file)
    
with open(p_0_training_File, 'rb') as file:  
    p_0 = pickle.load(file)

with open(V1_training_File, 'rb') as file:  
    V1_train = pickle.load(file)

## $3)$ Network training

### Mini-Batching and stochastic gradient descent

The sample is divided into mini-batch of size 256 to allow faster computation.

The insight of stochastic gradient descent is that the gradient is an expectation. The expectation may be approximately estimated using a small set of samples. Specifically, on each step of the algorithm, we can sample a minibatch of examples $\mathbb{B}=\left\{\boldsymbol{(S,V)}^{(1)}, \ldots, \boldsymbol{(S,V)}^{\left(m^{\prime}\right)}\right\}$ drawn uniformly from the training set. 


The estimate of the gradient is formed as
$$
g=\frac{1}{m^{\prime}} \nabla_{\theta} \sum_{i=1}^{m^{\prime}} L\left((S,V)^{(i)}, Z^{(i)}, \theta\right)
$$
using examples from the minibatch $\mathbb{B}$. 

The stochastic gradient descent algorithm then follows the estimated gradient downhill:
$$
\theta \leftarrow \theta-\epsilon g
$$
where $\epsilon$ is the learning rate.

Reference: Deep Learning, Ian Goodfellow Yoshua Bengio Aaron Courville, Chapter 5, p152

### Training

The loss function is defined as: 

$$L(S,V,Z,{\theta})= \mathbb{E}\left[\left(-Z+p_{0}+\left(\delta_S^{\theta} \cdot S\right)_{T}+\left(\delta_V^{\theta} \cdot V\right)_{T}\right)^{2}\right]$$

With the Mini-Batching the loss is therefore:

$$
L=\frac{1}{m}  \sum_{i=1}^{m} L\left((S,V)^{(i)}, Z^{(i)}, \theta\right)
$$

Where m is the number of mini-batchs

In [ ]:
def shuffle_batch(sampleS,sampleV, batch_size):
    rnd_idx = np.random.permutation(sampleS.shape[0])
    sample_batchS=[]
    sample_batchV=[]
    n_batches = sampleS.shape[0] // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
      sample_batchS.append(sampleS[batch_idx,:,:])
      sample_batchV.append(sampleV[batch_idx,:,:])
    return ( (sample_batchS,sample_batchV) )

In [ ]:
# Training data
S1_train500k=S1_train[:499968,:,:]
p_0_500k=p_0[:499968,:,:]
V1_train500k=V1_train[:499968,:,:]

test_S1=S1_train[500000:501000,:,:]
test_p_0=p_0[500000:501000,:,:]
test_V1=V1_train[500000:501000,:,:]

In [ ]:
# Training setting 

learning_rate=0.005
batch_size=256
epochs=200
nb_batch=S1_train500k.shape[0]//batch_size #nb of patch / set
optimizer = torch.optim.Adam(fRnn_LSTM.parameters(), lr=learning_rate)
loss = nn.MSELoss()

In [ ]:
min_loss=10
LOSS_B=[]
LOSS_test=[]
strat=[]
pnl_evol=[]
mediane=[]
rate_timer=0
training_rates=[learning_rate]

In [ ]:
## Training

t1=time()

shuffleS, shuffleV = shuffle_batch(S1_train500k,V1_train500k,batch_size)

for epoch in range(epochs):
    
    loss_b=0


    for sample_batchS, sample_batchV in zip(shuffleS, shuffleV):

        optimizer.zero_grad()

        input=torch.cat((sample_batchS, sample_batchV),dim=2)

        S=sample_batchS[:,1:,:]-sample_batchS[:,:-1,:]
        V=sample_batchV[:,1:,:]-sample_batchV[:,:-1,:]
        
        output = fRnn_LSTM(input)

        loss_batch= loss(torch.sum(S*output[:,:-1,:1],dim=1)+torch.sum(V*output[:,:-1,1:],dim=1)+p_0[:batch_size,0,:],torch.max(sample_batchS[:,-1,:]-K,torch.zeros(batch_size, 1)) )
         
        loss_b += loss_batch/nb_batch    
        
        loss_batch.backward()
        optimizer.step()
    
        min_loss=min(min_loss,loss_batch)
      
    LOSS_B.append(loss_b.item())
  

    

    print("Learning Rate", training_rates[-1] )
    print(colored("Epoch:",'red'), epoch,"loss:", colored(loss_b.item(),'red') )
    print( "Best Loss", min(LOSS_B))
    print('Best Batch Loss',min_loss , '\n')
    
    S=test_S1[:,1:,:]-test_S1[:,:-1,:]
    V=test_V1[:,1:,:]-test_V1[:,:-1,:]

    input=torch.cat((test_S1,test_V1),dim=2)
    output=fRnn_LSTM(input)
    x=torch.sum(S*output[:,:-1,:1],dim=1)+torch.sum(V*output[:,:-1,1:],dim=1)-torch.max(test_S1[:,-1,:]-K,torch.zeros(test_S1.shape[0], 1))+test_p_0[:,0,:]

    test=loss(torch.sum(S*output[:,:-1,:1],dim=1)+torch.sum(V*output[:,:-1,1:],dim=1)+test_p_0[:,0,:],torch.max(test_S1[:,-1,:]-K,torch.zeros(test_S1.shape[0], 1)))
    LOSS_test.append(test)
    
    if (test<=min(LOSS_test)):
        with open(fRnn_LSTM_model_File, 'wb') as file:  
            pickle.dump(fRnn, file)      
    
    strat.append(torch.mean(torch.sum(S*output[:,:-1,:1],dim=1)+torch.sum(V*output[:,:-1,1:],dim=1)).item())
    pnl_evol.append(torch.mean( x ).item())
    mediane.append(torch.median( x ).item())
    quadra=("Quadratic loss ", test.item())
    print(colored("TEST",'green') )
    print("Min Test Loss", min(LOSS_test))
    print("PnL moyen", torch.mean( x ).item())
    print(colored(quadra,'green'))
    print( "PnL median", torch.median( x ).item())
    print( "Strat moyenne", torch.mean(torch.sum(S*output[:,:-1,:1],dim=1)+torch.sum(V*output[:,:-1,1:],dim=1)).item())
    print( "Payoff - p_0 moyen", torch.mean(-torch.max(test_S1[:,-1,:]-K,torch.zeros(test_S1.shape[0], 1))+test_p_0[:,0,:]).item(), '\n')


t2=time()

print( colored('Finale Validation Loss','red') , colored(loss_b.item(),'red'))
print( colored('Best Validation Loss', 'green'), min(LOSS_B))
print(colored('Finale Test Loss', 'red'), LOSS_test[-1])
print(colored('Best Test Loss', 'green' ), min(LOSS_test) )
print('Data Size', S1_train500k.shape)
print("Training Time (h)", (t2-t1)/3600 ,'\n') 

print( "Evolution during training")


print( "Quadratic loss evolution")
plt.plot(LOSS_test,color='green')
plt.plot(LOSS_B,color='red')
plt.show()

print("Learning Rates")
plt.plot(training_rates)
plt.show()

print("strat test")
plt.plot(strat)
plt.show()
print( "PNL Test")
plt.plot(pnl_evol)
plt.show()

print("mediane Test")
plt.plot(mediane)
plt.show()

plt.hist(x.detach().numpy(),bins=50)

